In [6]:
import pandas as pd 
import ast
df = pd.read_csv("../datas/movies_metadata.csv")
df.head()

df.genres = df.genres.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df.genres = df.genres.apply(lambda x: [genre['name'] for genre in x])

C:\Users\feldm\AppData\Local\Temp\ipykernel_8484\1140485895.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../datas/movies_metadata.csv")


 Time to define important columns

for nlp :

- overview 
- Title
- vote_average

for similarities : 
- overview
- title
- genre

other : 

- vote_count
- popularity
- poster_path
- video
- genres
- release_date
- runtime


In [15]:
reduce = df[['genres', 'title', 'overview']]
reduce = reduce.dropna()
reduce = reduce.astype({'genres':'string'})
reduce.head()

,genres,title,overview
0,"['Animation', 'Comedy', 'Family']",Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,"['Adventure', 'Fantasy', 'Family']",Jumanji,When siblings Judy and Peter discover an encha...
2,"['Romance', 'Comedy']",Grumpier Old Men,A family wedding reignites the ancient feud be...
3,"['Comedy', 'Drama', 'Romance']",Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,['Comedy'],Father of the Bride Part II,Just when George Banks has recovered from his ...


In [16]:
def important_features(dataset):
    data = dataset.copy()
    for i in range(0,dataset.shape[0]):
        data["full"] = data['title'] + data['overview'] + data["genres"]
    return data

In [17]:
data = important_features(reduce)

KeyboardInterrupt: 

In [ ]:
data.head()

### Preprocessing for nlp

In [9]:
import re
import nltk
nltk.download('punkt')
from nltk import word_tokenize

def clean_text(text):
  text = text.lower()
  text = re.sub("[^a-zA-Z\'\-]", " ", text) 
  return " ".join(word_tokenize(text)[:256])

data["full"] = data['full'].apply(clean_text)
data.head()

In [20]:
reduce_1000 = reduce.head(2000)

### TF-IDF 

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
  
# Vectorizing pre-processed movie plots using TF-IDF
tfidfvec = TfidfVectorizer()
tfidf_movieid = tfidfvec.fit_transform((reduce_1000["overview_processed"]))
  
# Finding cosine similarity between vectors
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfidf_movieid, tfidf_movieid)

In [22]:
# Storing indices of the data
indices = pd.Series(reduce_1000.index)
  
def recommendations(title, cosine_sim = cos_sim):
    recommended_movies = []
    index = indices[indices == title].index[0]
    similarity_scores = pd.Series(cosine_sim[index]).sort_values(ascending = False)
    top_5_movies = list(similarity_scores.iloc[1:6].index)
    for i in top_5_movies:
        recommended_movies.append(list(reduce_1000.index)[i])
    return recommended_movies

In [28]:
recommendations('Toy Story')

['Toy Story 2',
 'Man on the Moon',
 'Rebel Without a Cause',
 'Malice',
 'Condorman',
 'For Love or Money',
 "Child's Play 3",
 "Losin' It",
 'Bound for Glory',
 'The Sunchaser']